In [1]:
import pandas as pd
import requests
import yaml

In [2]:
# Load credential data from yaml-file
with open("./credentials.yml", "r") as credentials:
    try:
        credentials_data = yaml.safe_load(credentials)
        key = credentials_data['rapidsAPI']['API-KEY']
        host = credentials_data['rapidsAPI']['API-HOST']
    except:
        print('Error')

In [3]:
# Function to create payload
def create_payload(country,start,end):
    parameter = {
        'country_code': country,
        'start_date': start,
        'end_date': end
    }
    
    return parameter

In [4]:
url = 'https://working-days.p.rapidapi.com/1.3/analyse'

headers = {
    "X-RapidAPI-Key": key,
    "X-RapidAPI-Host": host
}

In [6]:
payload = create_payload('DE','2013-01-01','2013-01-31')

In [6]:
response = requests.get(url, headers=headers, params=payload)
jsn_response = response.json()

In [7]:
# Extract List of public holidays from JSON
lst_public_holidays = jsn_response['public_holidays']['list']

# Create DataFrame from List
df_public_holidays = pd.DataFrame(lst_public_holidays)

In [8]:
# Check DataFrame
df_public_holidays

,date,description
0,2013-01-01,Neujahrstag


In [7]:
# Save DataFrame to CSV-File
df_public_holidays.to_csv('./src/output/api_public_holidays.csv', index=False)

In [17]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///./src/output/workingDays', echo=False)
db_response = df_public_holidays.to_sql(con=engine, name="public_holidays", index=False, if_exists="replace")
db_response

1